<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import random 
import re
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.layers import Dense, Embedding, Flatten ,MultiHeadAttention,Dropout
# MultiHeadAttention

In [ ]:
## 1) Sentences will be tokenized using the TextVectorization 
## 2) 12.5 % of the tokens will be replaced with the [MASK] token. Substituded tokens will be 
## chosen randomly. To select the random tokens, random sampling will be chosen 

## 3) segment id will be determined at the second step
## 4) positional embeddings will be directly taken from the article

In [ ]:
max_length =  128  ## 256
corpus_length = 128   ## 30000
embedding_dimension = 128   ## 128
vocab_size = 128  ## 30000
number_heads = 8



In [ ]:
trial_sentences = [
    'my name is john',
    'london is the capital is the england',
    'weather is cold at the northern hemisphere of the world',
    'today, i will go the school'
]

longest_sentence = []
for i in trial_sentences:
    longest_sentence.append(len(i.split()))
    
print(np.max(longest_sentence))

longest_sentence = np.max(longest_sentence)

10


In [ ]:
def masking_procedure (text,tok):
    
    # tf_tokenizer = Tokenizer()
    tf_tokenizer = tok
    tf_tokens=tf_tokenizer.fit_on_texts(text)

    corpus = tf_tokenizer.word_index  ### TOKEN SET WITH TOKEN IDS WITHOUT THE SPECIAL TOKENS 
    extended_corpus = corpus.copy()   ### TOKEN SET WITH SPECIAL TOKENS

    extended_corpus['MASK'] = len(corpus)+1

    corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
    for i,j in enumerate(corpus):
        corpus_tokens.append(j)
#         print(i,j)

        
    extended_tokens = []
    for i,j in  enumerate(extended_corpus):
    #     print(i,j)
        extended_tokens.append(j)
    # print('maskelenmis_corpus')
    # print('-------')
    subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12)) ## RANDOMLY SELECTED TOKENS TO BE MASKED
#     print(subs_tokens)

    sequences = tf_tokenizer.texts_to_sequences(text)
    padded_sequences= pad_sequences(sequences)
    padded_sequences

    substitute_padded_sequences= padded_sequences.copy()
    actual_padded_sequences = padded_sequences.copy()

    subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

    for i in subs_tokens:
        subs_tokens_ids.append(tf_tokenizer.word_index[i])
    
    
    mask_token_id = 20
#     print(subs_tokens_ids)



    for i in range(len(extended_tokens)):
        if i == (subs_tokens_ids[0]-1   ):
            extended_tokens[i] = '[MASK]'
    
        if i == (subs_tokens_ids[1]-1):
            extended_tokens[i] = '[MASK]'
        
        
        
    maskeli = []
    for i in range(substitute_padded_sequences.shape[0]):
        for j in range(substitute_padded_sequences.shape[1]):
            if substitute_padded_sequences[i][j] != 0:
                x = (substitute_padded_sequences[i][j]  )
                maskeli.append("".join(extended_tokens[x-1]))
                
                
    maskeli_2 = []
    for i in (substitute_padded_sequences):
        maskeli_2.append('\n')
        for j in range(len(i)):
        
            if (i[j] !=0):
#             print(i[j])
                maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
    y= " ".join(maskeli_2)

    y = y.split('\n')
    y = np.delete(y,0,0)
    y
     
    segment_ids = substitute_padded_sequences.copy()

    for i in range(segment_ids.shape[0]):
        for j in range(segment_ids.shape[1]):
            if (segment_ids[i][j] != 0):
                segment_ids[i][j] = 1
    segment_ids = np.array(segment_ids)
    
    return (y , extended_tokens, segment_ids,substitute_padded_sequences,actual_padded_sequences)

In [ ]:
tf_tokenizer = Tokenizer()

In [ ]:
masked_text, masked_tokens,segment_ids , masked_sequence, original_sequences = masking_procedure(trial_sentences,tf_tokenizer)

In [ ]:
masked_sequence,original_sequences,masked_text

(array([[ 0,  0,  0,  0,  0,  0,  3,  4,  2,  5],
        [ 0,  0,  0,  6,  2,  1,  7,  2,  1,  8],
        [ 9,  2, 10, 11,  1, 12, 13, 14,  1, 15],
        [ 0,  0,  0,  0, 16, 17, 18, 19,  1, 20]], dtype=int32),
 array([[ 0,  0,  0,  0,  0,  0,  3,  4,  2,  5],
        [ 0,  0,  0,  6,  2,  1,  7,  2,  1,  8],
        [ 9,  2, 10, 11,  1, 12, 13, 14,  1, 15],
        [ 0,  0,  0,  0, 16, 17, 18, 19,  1, 20]], dtype=int32),
 array([' my [MASK] is john ', ' [MASK] is the capital is the england ',
        ' weather is cold at the northern hemisphere of the world ',
        ' today i will go the school'], dtype='<U57'))

In [ ]:
MAX_LEN = 256
EMBED_DIM = 128



def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc


positional_embeddings = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)

In [ ]:
pos_enc = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)
# pos_enc + segment_ids

total_encoding = masked_sequence+segment_ids # +pos_enc  (bi sekilde positional encodingi sokmam gerek )

In [ ]:
total_encoding.shape
intro_shape = total_encoding.shape

In [ ]:
# input_layer = layers.Input(shape = (10,))
# emb_layer = layers.Embedding(intro_shape[0],embedding_dimension , input_length = max_length)(input_layer)
# att_layer = layers.MultiHeadAttention(num_heads= number_heads , key_dim =embedding_dimension)(emb_layer,emb_layer,emb_layer)
# dropout1_layer = layers.Dropout(0.1)(att_layer)
# norm_layer_1 = layers.LayerNormalization(
#         epsilon=1e-6
#     )(emb_layer + dropout1_layer)
# ### After this point feed forward nn will be constructed with dense layers

# d1_layer = layers.Dense(128, activation = 'relu')(norm_layer_1)
# # d2_layer = layers.Dense(embedding_dimension)

# dropout2_layer = layers.Dropout(0.1)(d1_layer)

# output_layer = layers.LayerNormalization(epsilon=1.06)(norm_layer_1 +d1_layer)


# tf_mlm_model = keras.Model(inputs =input_layer, outputs = output_layer )
# tf_mlm_model.summary()
# keras.utils.plot_model(tf_mlm_model)


# deneme_loss = tf.keras.losses.SparseCategoricalCrossentropy()


# tf_mlm_model.compile(
    
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam'
# )

# tf_mlm_model.fit(masked_sequence, original_sequences, epochs =5)

In [ ]:
# input_layer = layers.Input(shape = (10,))
# l1_layer = layers.Embedding(corpus_length, 10, input_length =10)(input_layer)
# # l1_layer = layers.Flatten()(l1_layer)
# out_layer  = layers.Dense(10, activation= 'softmax')(l1_layer)
# tf_trial_model  = keras.Model(inputs= input_layer, outputs = out_layer)

# tf_trial_model.compile(
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam',
#     metrics = ['accuracy']
# )

# tf_trial_model.fit(
#                   #  np.transpose(masked_sequence),
#                    masked_sequence, 
#                    original_sequences, 
#                    epochs = 100)

In [ ]:
dink = layers.Embedding(300, 5)(masked_sequence)

In [ ]:
dink.shape

TensorShape([4, 10, 5])

In [ ]:
dink[1].shape

TensorShape([10, 5])

In [ ]:
deneme_matrisi = np.array([[2],[4]])
deneme_matrisi_2 = np.array([[1,2,3,4],
                             [5,6,7,8]])
deneme_matrisi.shape

len(deneme_matrisi)

2

In [ ]:
x = layers.Embedding(21,2,)(deneme_matrisi)
print(x)

tf.Tensor(
[[[-0.04596917 -0.04789708]]

 [[-0.03241487 -0.01382715]]], shape=(2, 1, 2), dtype=float32)


In [ ]:
layers.Dense(1,activation = 'softmax')(x)

<tf.Tensor: shape=(2, 1, 1), dtype=float32, numpy=
array([[[1.]],

       [[1.]]], dtype=float32)>

In [ ]:
layers.Embedding(corpus_length, 1, input_length =10)(masked_sequence)

<tf.Tensor: shape=(4, 10, 1), dtype=float32, numpy=
array([[[-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [ 0.04986315],
        [ 0.04745692],
        [-0.01612135],
        [-0.00112344]],

       [[-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [-0.04053469],
        [-0.01612135],
        [-0.03321778],
        [ 0.04913745],
        [-0.01612135],
        [-0.03321778],
        [ 0.04846687]],

       [[-0.00867294],
        [-0.01612135],
        [-0.02571096],
        [-0.03978658],
        [-0.03321778],
        [-0.04569636],
        [ 0.02915999],
        [ 0.02300172],
        [-0.03321778],
        [-0.0275734 ]],

       [[-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [-0.03723358],
        [ 0.02888061],
        [-0.0422257 ],
        [-0.01365086],
        [ 0.03446329],
        [-0.03321778],
        [-0.03740649]]], dtype=float32)>